In [11]:
import pandas as pd

# Load the dataset
file_path = 'processed_train.csv'
df = pd.read_csv(file_path)

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Define predictor variables (X) and the target variable (y)
X = df.drop('imdb_score_binned', axis=1)
y = df['imdb_score_binned']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Initialize and train the dummy classifier (0R)
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dummy_clf.predict(X_test)

# Calculate the accuracy of the dummy classifier
accuracy = accuracy_score(y_test, y_pred)

# Calculate additional metrics: f1 score, precision, and recall
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

accuracy, f1, precision, recall

/Users/hkharith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.610648918469218, 0.4630333741285186, 0.37289210162762565, 0.610648918469218)

In [5]:
#!pip install xgboost #remove comment to install if needed

In [13]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
import numpy as np

# Define the stratified K-fold cross-validator
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(random_state=1, use_label_encoder=False, eval_metric='mlogloss')

# Define scoring metrics
scoring_metrics = {'accuracy': make_scorer(accuracy_score),
                   'f1': make_scorer(f1_score, average='weighted'),
                   'precision': make_scorer(precision_score, average='weighted'),
                   'recall': make_scorer(recall_score, average='weighted')}

# Perform stratified K-fold cross-validation and calculate metrics
cross_val_metrics = {metric: cross_val_score(xgb_clf, X, y, cv=stratified_kfold, scoring=scorer)
                     for metric, scorer in scoring_metrics.items()}

# Calculate and display the average of each metric
average_metrics = {metric: np.mean(scores) for metric, scores in cross_val_metrics.items()}
average_metrics

/Users/hkharith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkharith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkharith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkharith/anaconda3/lib/python3.11/site-packages/skl

{'accuracy': 0.7350221852468108,
 'f1': 0.7001067241805645,
 'precision': 0.7134487345261303,
 'recall': 0.7350221852468108}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize the GridSearchCV object with stratified K-fold
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=stratified_kfold, scoring='accuracy', verbose=1)

# Perform grid search
grid_search.fit(X, y)

# Get the best score and parameters
best_score = grid_search.best_score_
best_params = grid_search.best_params_

(best_score, best_params)

In [14]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# Load the dataset
file_path = './processed_train.csv'
data = pd.read_csv(file_path)

# Prepare features and target
X = data.drop('imdb_score_binned', axis=1)
y = data['imdb_score_binned']

# Define the model with hyperparameters
model = XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9)

# Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

fold_performance = []

# Perform cross-validation
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    fold_performance.append(score)

# Output the performance
fold_performance

[0.7437603993344426,
 0.7720465890183028,
 0.7371048252911814,
 0.7138103161397671,
 0.7533333333333333]

In [15]:
# Fit the model on the full dataset
model.fit(X, y)

full_dataset_score = model.score(X, y)

full_dataset_score

0.8861517976031957

In [16]:
# Load the test dataset
file_path_test = 'processed_test_aligned.csv'
test_data = pd.read_csv(file_path_test)

# Prepare features for prediction
X_test = test_data.drop('imdb_score_binned', axis=1)

# Predict on the test dataset
predictions = model.predict(X_test)

# Add predictions to the test dataframe
test_data['predicted_score_binned'] = predictions

# Output the first few rows of the test dataframe with predictions
test_data[['id', 'predicted_score_binned']].head()

,id,predicted_score_binned
0,1,2
1,2,2
2,3,2
3,4,3
4,5,2


In [17]:
predicted_output = test_data[['id', 'predicted_score_binned']]
predicted_output.columns = ['id', 'imdb_score_binned']
predicted_output.head()
predicted_output.to_csv('xgbscorefin.csv',index = False)

In [1]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y, y_pred_full)

fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=[str(i) for i in range(conf_matrix.shape[1])],
    y=[str(i) for i in range(conf_matrix.shape[0])],
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(
    autosize=False,
    width=500,  
    height=500,  
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)

fig.show()

NameError: name 'y' is not defined

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate evaluation metrics
accuracy = accuracy_score(y, y_pred_full)
precision = precision_score(y, y_pred_full, average='weighted')
recall = recall_score(y, y_pred_full, average='weighted')
f1 = f1_score(y, y_pred_full, average='weighted')

metrics = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

metrics

NameError: name 'y' is not defined

In [ ]:
from sklearn.model_selection import learning_curve
import numpy as np
import plotly.graph_objects as go

# Define the number of training examples to use for learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

fig = go.Figure()

# Calculate the learning curve
train_sizes, train_scores, validation_scores = learning_curve(
    estimator=model,
    X=X,
    y=y,
    train_sizes=train_sizes,
    cv=5,
    scoring='accuracy'
)

fig.add_trace(go.Scatter(x=train_sizes, y=train_scores_mean, name='Training score',
                         mode='lines+markers', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=train_sizes, y=validation_scores_mean, name='Cross-validation score',
                         mode='lines+markers', line=dict(color='red')))

fig.update_layout(xaxis_title='Training examples', yaxis_title='Score',
                  legend=dict(yanchor='top', y=0.99, xanchor='right', x=1.2),
                  autosize=False, width=600, height=600)

fig.show()

In [ ]:
from sklearn.model_selection import validation_curve
import xgboost as xgb
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define the range of max_depth values
param_range = np.arange(3, 7)

# Calculate validation curve
train_scores, test_scores = validation_curve(
    model, X, y, param_name='max_depth', param_range=param_range,
    cv=5, scoring='accuracy', n_jobs=-1)

# Calculate mean and standard deviation for training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot validation curve
fig = make_subplots(rows=1, cols=1)

# Add the training score curve
fig.add_trace(go.Scatter(x=param_range, y=train_mean, name='Training score',
                         mode='lines+markers', line_color='blue'),
              row=1, col=1)

# Add the test score curve
fig.add_trace(go.Scatter(x=param_range, y=test_mean, name='Cross-validation score',
                         mode='lines+markers', line_color='orange'),
              row=1, col=1)

# Update xaxis properties
fig.update_xaxes(title_text='max_depth', row=1, col=1)
# Update yaxis properties
fig.update_yaxes(title_text='Accuracy', row=1, col=1)
# Update layout to make the plot square
fig.update_layout(autosize=False, width=600, height=600)  # Adjust width and height to make the plot square

# Show plot
fig.show()